In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from transformers import ViTForImageClassification, ViTConfig
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split


1. CHUẨN BỊ DATASET

In [ ]:
class ActionDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        # Lấy danh sách các lớp từ tên thư mục
        self.classes = sorted(os.listdir(root_dir))
        # Tạo dictionary ánh xạ tên lớp -> số index
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        # Load tất cả đường dẫn ảnh và nhãn tương ứng
        self.images = self._load_images()

    def _load_images(self):
        #Hàm helper để load tất cả ảnh và nhãn
        images = []
        for class_name in self.classes:
            class_dir = os.path.join(self.root_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                # Lưu cặp (đường dẫn ảnh, index lớp)
                images.append((img_path, self.class_to_idx[class_name]))
        return images

    def __len__(self):
        #Trả về số lượng ảnh trong dataset"""
        return len(self.images)

    def __getitem__(self, idx):
        #Lấy ảnh và nhãn tại vị trí idx
        img_path, label = self.images[idx]
        # Mở ảnh và chuyển sang RGB (phòng trường hợp ảnh grayscale)
        image = Image.open(img_path).convert('RGB')

        # Áp dụng các phép biến đổi nếu có
        if self.transform:
            image = self.transform(image)

        return image, label


2. ĐỊNH NGHĨA TRANSFORM

In [ ]:
# Biến đổi ảnh cho tập huấn luyện (data augmentation mạnh để tăng khả năng khái quát)
train_transform = transforms.Compose([
    transforms.Resize(256),  # Resize ảnh về kích thước 256 (giữ nguyên tỷ lệ cạnh dài)

    transforms.RandomResizedCrop(224, scale=(0.6, 1.0)),  # Cắt ngẫu nhiên vùng 224x224 từ ảnh gốc đã resize
    # scale=(0.6, 1.0): tỉ lệ vùng cắt so với ảnh gốc từ 60% đến 100%
    # -> tạo sự đa dạng về kích thước và vị trí ảnh input

    transforms.RandomHorizontalFlip(p=0.5),  # Lật ngang ảnh ngẫu nhiên 50%
    # Giúp mô hình học được tính bất biến khi đối tượng bị lật

    transforms.RandomVerticalFlip(p=0.2),  # Lật dọc ảnh với xác suất 20%
    # Ít phổ biến hơn nhưng vẫn có ích trong một số bài toán (ví dụ: tay, yoga...)

    transforms.RandomApply(
        [transforms.ColorJitter(0.4, 0.4, 0.4, 0.2)], p=0.8
    ),
    # Áp dụng biến đổi màu sắc (ColorJitter) với xác suất 80%
    # Điều chỉnh độ sáng, tương phản, bão hòa và hue ngẫu nhiên
    # -> giúp mô hình chống lại sự thay đổi ánh sáng môi trường

    transforms.RandomGrayscale(p=0.2),  # Chuyển ảnh thành grayscale (đen trắng) với xác suất 20%
    # Bắt buộc mô hình học cả đặc trưng hình dạng mà không phụ thuộc màu sắc

    transforms.RandomRotation(15),  # Xoay ảnh ngẫu nhiên trong khoảng ±15 độ
    # Tạo ảnh biến thể về góc nhìn, tăng khả năng tổng quát

    transforms.ToTensor(),  # Chuyển ảnh PIL thành tensor (C x H x W) và chuẩn hóa pixel [0, 1]

    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # Trung bình kênh RGB (ImageNet)
        std=[0.229, 0.224, 0.225]    # Độ lệch chuẩn kênh RGB (ImageNet)
    ),
    # Chuẩn hóa giá trị ảnh theo phân phối của ImageNet — quan trọng vì mô hình ViT pretrained trên ImageNet
])

# Biến đổi ảnh cho tập validation và test (không dùng augmentation để giữ đánh giá chính xác)
test_transform = transforms.Compose([
    transforms.Resize(256),  # Resize chiều dài ngắn nhất về 256
    transforms.CenterCrop(224),  # Cắt vùng trung tâm kích thước 224x224
    # Đảm bảo input có cùng kích thước với ảnh khi train mà không thêm biến thể

    transforms.ToTensor(),  # Chuyển ảnh thành tensor: mảng nhiều chiều
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # Trung bình RGB (ImageNet)
        std=[0.229, 0.224, 0.225]    # Độ lệch chuẩn RGB (ImageNet)
    )
])


3. TẠO DATA DATASET VÀ KIỂM TRA PHÂN BỐ LỚP

In [ ]:
# 1. Khởi tạo datasets với transform tương ứng
train_dataset = ActionDataset(
    '/content/drive/MyDrive/Colab Notebooks/Final/Data/train_data',
    transform=train_transform  # Áp dụng augment mạnh khi training
)

test_dataset = ActionDataset(
    '/content/drive/MyDrive/Colab Notebooks/Final/Data/test_data',
    transform=test_transform  # Resize + crop cố định khi test
)

# 2. Kiểm tra phân bố lớp ban đầu trong tập train và test
print("Phân bố lớp tập train ban đầu:",
      Counter([label for _, label in train_dataset.images]))  # Đếm số lượng ảnh mỗi lớp trong train

print("Phân bố lớp tập test:",
      Counter([label for _, label in test_dataset.images]))   # Đếm số lượng ảnh mỗi lớp trong test

# 3. Tách validation set từ train_dataset (tách 20% ảnh để làm tập val)
indices = list(range(len(train_dataset)))  # Danh sách chỉ số ảnh: [0, 1, 2, ..., N-1]

# Sử dụng stratify để đảm bảo phân bố lớp giữa train/val là giống nhau
train_idx, val_idx = train_test_split(
    indices,
    test_size=0.2,               # 20% dùng làm validation
    random_state=42,             # Đảm bảo kết quả tách luôn giống nhau khi chạy lại
    stratify=[label for _, label in train_dataset.images]  # Giữ phân bố lớp
)

# Tạo các tập con (subset) cho train và val
train_subset = torch.utils.data.Subset(train_dataset, train_idx)
val_subset = torch.utils.data.Subset(train_dataset, val_idx)

# 4. Hàm hỗ trợ để đếm lại phân bố lớp trong các subset
def get_label_counts(subset):
    return Counter([subset.dataset.images[i][1] for i in subset.indices])

# In ra phân bố lớp sau khi tách train/val
print("\nSau khi tách:")
print("Phân bố lớp tập train:", get_label_counts(train_subset))
print("Phân bố lớp tập val:", get_label_counts(val_subset))

# 5. Gán lại transform cho val (dùng transform giống test để không làm méo ảnh)
# Vì val_subset vẫn dùng chung dataset gốc (train_dataset), nên cần cập nhật lại transform cho val
val_subset.dataset.transform = test_transform


Phân bố lớp tập train ban đầu: Counter({0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000, 10: 1000, 11: 1000, 12: 1000, 13: 1000, 14: 1000})
Phân bố lớp tập test: Counter({0: 200, 1: 200, 2: 200, 3: 200, 4: 200, 5: 200, 6: 200, 7: 200, 8: 200, 9: 200, 10: 200, 11: 200, 12: 200, 13: 200, 14: 200})

Sau khi tách:
Phân bố lớp tập train: Counter({5: 800, 11: 800, 0: 800, 4: 800, 6: 800, 10: 800, 8: 800, 9: 800, 3: 800, 7: 800, 14: 800, 1: 800, 2: 800, 13: 800, 12: 800})
Phân bố lớp tập val: Counter({9: 200, 5: 200, 13: 200, 11: 200, 3: 200, 0: 200, 1: 200, 6: 200, 8: 200, 4: 200, 7: 200, 10: 200, 14: 200, 12: 200, 2: 200})


4. TẠO DATA LOADERS

In [ ]:
from torch.utils.data import DataLoader

# 1. Tạo DataLoader cho tập train
train_loader = DataLoader(
    train_subset,       # Sử dụng tập con đã chia từ train_dataset
    batch_size=32,      # Số lượng ảnh xử lý mỗi batch
    shuffle=True,       # Trộn dữ liệu mỗi epoch để tránh overfitting
    num_workers=2,      # Số tiến trình phụ đọc dữ liệu song song (tùy máy)
    pin_memory=True,    # Giúp tăng tốc khi truyền dữ liệu sang GPU
    persistent_workers=False  # Có thể đặt True nếu bạn chạy nhiều epoch trong môi trường ổn định
)

# 2. Tạo DataLoader cho tập validation
val_loader = DataLoader(
    val_subset,
    batch_size=32,
    shuffle=False,      # Không cần shuffle để giữ thứ tự kiểm định
    num_workers=2,
    pin_memory=True
)

# 3. Tạo DataLoader cho tập test
test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,      # Không cần shuffle trong test
    num_workers=2,
    pin_memory=True
)


5. XÂY DỰNG MODEL VISION TRANSFORMER


In [ ]:
# Số lượng lớp đầu ra cần phân loại (phù hợp với bài toán của bạn)
num_classes = 15

# 1. Tải cấu hình (config) của mô hình ViT gốc từ HuggingFace
config = ViTConfig.from_pretrained('google/vit-base-patch16-224')

# 2. Cập nhật số lượng lớp đầu ra phù hợp với số lớp trong bài toán của bạn
config.num_labels = num_classes  # Cập nhật số lớp output classifier

# 3. Cấu hình dropout để giảm overfitting (nếu dữ liệu nhỏ, nên tăng một chút)
config.hidden_dropout_prob = 0.3               # Dropout tại các lớp MLP (Feed-forward layers)
config.attention_probs_dropout_prob = 0.3      # Dropout tại attention layers

# 4. Khởi tạo mô hình ViT với trọng số pretrained từ mô hình gốc
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224',  # Sử dụng mô hình ViT được pretrain trên ImageNet-21k
    config=config,                  # Áp dụng config đã chỉnh sửa phía trên
    ignore_mismatched_sizes=True    # Bỏ qua lỗi nếu kích thước lớp cuối (head) không khớp — do ta đổi số lớp
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([15]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([15, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


6. HÀM HỖ TRỢ

In [ ]:
# Tính độ chính xác của 1 patch
def calculate_accuracy(y_pred, y_true): #y_pred: giá trị dự đoán, y_true: nhãn thật
    _, predicted = torch.max(y_pred, 1)  # Lấy chỉ số lớp có xác suất cao nhất
    correct = (predicted == y_true).sum().item()  # Đếm số lượng dự đoán đúng
    return correct / y_true.size(0)  # Tính độ chính xác (accuracy) = đúng / tổng
#Đánh giá mô hình trên tập Validation/test
def evaluate(model, dataloader):
    model.eval()  # Đưa mô hình về chế độ đánh giá (tắt dropout, batchnorm,...)
    total_loss, total_acc = 0.0, 0.0  # Biến lưu tổng loss và accuracy
    all_preds, all_labels = [], []   # Danh sách lưu toàn bộ dự đoán và nhãn thật

    with torch.no_grad():  # Không tính gradient (giúp tiết kiệm bộ nhớ, tăng tốc)
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)  # Đưa dữ liệu lên GPU (nếu có)
            outputs = model(images)  # Dự đoán đầu ra qua mô hình
            loss = criterion(outputs.logits, labels)  # Tính loss giữa dự đoán và nhãn thật

            total_loss += loss.item()  # Cộng dồn loss
            total_acc += calculate_accuracy(outputs.logits, labels)  # Tính và cộng dồn độ chính xác

            # Lưu lại toàn bộ nhãn dự đoán và nhãn thật để đánh giá sau
            all_preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())  # Dự đoán lớp
            all_labels.extend(labels.cpu().numpy())  # Nhãn thật

    # Trả về:
    # - Loss trung bình
    # - Accuracy trung bình
    # - Danh sách dự đoán tất cả ảnh
    # - Danh sách nhãn thật tất cả ảnh
    return (total_loss / len(dataloader),
            total_acc / len(dataloader),
            all_preds,
            all_labels)


7. THIẾT LẬP TRAINNING

In [ ]:
# Chọn thiết bị tính toán (nếu có GPU thì dùng, không thì dùng CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Đang sử dụng thiết bị: {device}")

# Đưa mô hình lên thiết bị (GPU hoặc CPU)
model = model.to(device)

# Hàm mất mát (Loss function) dùng cho phân loại nhiều lớp
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
# Trình tối ưu hóa AdamW với learning rate nhỏ để fine-tune mô hình pretrained
optimizer = optim.AdamW(
    model.parameters(),    # Các tham số sẽ được tối ưu
    lr=3e-5,               # Learning rate nhỏ vì đang fine-tune mô hình đã được pretrained
    weight_decay=0.01      # Regularization (chuẩn hóa) để tránh overfitting
)
# Scheduler giúp điều chỉnh learning rate trong quá trình huấn luyện
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',         # Theo dõi metric có xu hướng tăng (accuracy)
    patience=2,         # Nếu val_acc không tăng sau 2 epochs → giảm LR
    factor=0.5,         # Giảm learning rate còn 50%
    verbose=True        # In thông tin khi learning rate được giảm
)

# Số epoch huấn luyện
num_epochs = 15
# Biến lưu kết quả tốt nhất để dùng cho Early Stopping
best_val_acc = 0.0       # Lưu best accuracy trên tập validation
patience = 3             # Dừng sớm nếu val_acc không cải thiện sau 3 epochs
no_improve = 0           # Đếm số epoch liên tiếp không cải thiện

# Danh sách lưu lại loss và accuracy mỗi epoch (để vẽ biểu đồ sau)
train_losses, val_losses = [], []
train_accs, val_accs = [], []


Đang sử dụng thiết bị: cuda


8. VÒNG LẶP TRAINING

In [ ]:
for epoch in range(num_epochs):
    # ======= PHASE 1: HUẤN LUYỆN =========
    model.train()  # Đặt model vào chế độ huấn luyện (training mode)
    epoch_train_loss, epoch_train_acc = 0.0, 0.0  # Biến lưu tổng loss và acc của epoch

    # Hiển thị progress bar với tqdm để dễ theo dõi tiến trình huấn luyện
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Đưa dữ liệu lên GPU nếu có

        optimizer.zero_grad()  # Reset gradient trước mỗi batch
        outputs = model(images)  # Dự đoán đầu ra từ mô hình
        loss = criterion(outputs.logits, labels)  # Tính loss
        loss.backward()  # Lan truyền gradient ngược

        # Cắt gradient để tránh exploding gradients (giữ ổn định khi fine-tuning)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()  # Cập nhật trọng số

        # Tính độ chính xác và cộng dồn loss + acc cho cả epoch
        acc = calculate_accuracy(outputs.logits, labels)
        epoch_train_loss += loss.item()
        epoch_train_acc += acc

        # Cập nhật hiển thị tiến trình với loss và acc hiện tại
        progress_bar.set_postfix({'loss': loss.item(), 'acc': acc})


    # Trung bình loss và acc của cả epoch trên tập train
    avg_train_loss = epoch_train_loss / len(train_loader)
    avg_train_acc = epoch_train_acc / len(train_loader)

    # Đánh giá trên tập validation (tách riêng ở bước chuẩn bị dữ liệu)
    avg_val_loss, avg_val_acc, val_preds, val_labels = evaluate(model, val_loader)

    # Cập nhật scheduler (giảm LR nếu val_loss không cải thiện)
    scheduler.step(avg_val_loss)

    # Lưu lại loss và acc của epoch này
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    train_accs.append(avg_train_acc)
    val_accs.append(avg_val_acc)


    # In thông tin sau mỗi epochs
    print(f'\nEpoch {epoch+1}/{num_epochs}:')
    print(f'Train Loss: {avg_train_loss:.4f} | Train Acc: {avg_train_acc:.4f}')
    print(f'Val Loss: {avg_val_loss:.4f} | Val Acc: {avg_val_acc:.4f}')

    # Early stopping và lưu model tốt nhất
    if avg_val_acc > best_val_acc:
        best_val_acc = avg_val_acc
        no_improve = 0 # Reset bộ đếm early stopping
        model_path = '/content/drive/MyDrive/Colab Notebooks/Final/best_vit_model.pth'
        torch.save({
            'model_state_dict': model.state_dict(),            # Trọng số mô hình
            'optimizer_state_dict': optimizer.state_dict(),    # Trạng thái optimizer (dùng nếu muốn tiếp tục train)
            'epoch': epoch,                                    # Epoch hiện tại
            'best_val_acc': best_val_acc                       # Độ chính xác val tốt nhất
        }, model_path)
        print(f'Best model saved with val acc: {best_val_acc:.4f}')
    else:
        no_improve += 1 # Tăng bộ đếm nếu không cải thiện
        if no_improve >= patience:
            print(f"No improvement for {patience} epochs. Early stopping!")
            break # Dừng huấn luyện sớm nếu không cải thiện trong vài epoch liên tiếp

# Đánh giá cuối cùng trên test set
# Load lại mô hình có val acc cao nhất để đánh giá trên test
model.load_state_dict(torch.load(model_path)['model_state_dict'])
# Gọi hàm evaluate để tính test loss, accuracy và lưu lại các dự đoán
test_loss, test_acc, test_preds, test_labels = evaluate(model, test_loader)
# In ra độ chính xác cuối cùng và bảng phân loại
print(f'\nFinal Test Accuracy: {test_acc:.4f}')
print('Classification Report:')
print(classification_report(test_labels, test_preds, target_names=train_dataset.classes))

Epoch 1/15: 100%|██████████| 375/375 [59:17<00:00,  9.49s/it, loss=1.33, acc=0.688]



Epoch 1/15:
Train Loss: 1.7350 | Train Acc: 0.5364
Val Loss: 1.1994 | Val Acc: 0.7735
Best model saved with val acc: 0.7735


Epoch 2/15: 100%|██████████| 375/375 [07:03<00:00,  1.13s/it, loss=0.924, acc=0.875]



Epoch 2/15:
Train Loss: 1.2067 | Train Acc: 0.7473
Val Loss: 1.0576 | Val Acc: 0.8096
Best model saved with val acc: 0.8096


Epoch 3/15: 100%|██████████| 375/375 [07:02<00:00,  1.13s/it, loss=0.868, acc=0.875]


...


Epoch 7/15:
Train Loss: 0.8191 | Train Acc: 0.9056
Val Loss: 0.9821 | Val Acc: 0.8383


Epoch 8/15: 100%|██████████| 375/375 [06:43<00:00,  1.08s/it, loss=0.981, acc=0.844]



Epoch 8/15:
Train Loss: 0.7796 | Train Acc: 0.9217
Val Loss: 1.0095 | Val Acc: 0.8357


Epoch 9/15: 100%|██████████| 375/375 [06:43<00:00,  1.08s/it, loss=0.719, acc=0.938]



Epoch 9/15:
Train Loss: 0.7644 | Train Acc: 0.9265
Val Loss: 1.0038 | Val Acc: 0.8340
No improvement for 3 epochs. Early stopping!


LOAD MODEL ĐÃ TRAIN


In [ ]:
# 1. Khởi tạo device (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Đang sử dụng thiết bị: {device}")

# 2. Khởi tạo lại model với cùng cấu hình ban đầu
num_classes = 15
config = ViTConfig.from_pretrained('google/vit-base-patch16-224')
config.num_labels = num_classes
config.hidden_dropout_prob = 0.3
config.attention_probs_dropout_prob = 0.3

model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224',
    config=config,
    ignore_mismatched_sizes=True
).to(device)

# 3. Load trọng số đã lưu
model_path = '/content/drive/MyDrive/Colab Notebooks/Final/best_vit_model.pth'
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

# 4. Load optimizer state nếu cần (cho việc tiếp tục training)
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# 5. Đưa model vào chế độ evaluation
model.eval()

# 6. Kiểm tra model đã load
print("\nModel đã được load thành công từ:", model_path)
print(f"Best validation accuracy khi train: {checkpoint['best_val_acc']:.4f}")
print(f"Epoch cuối cùng được train: {checkpoint['epoch'] + 1}")

# 7. Kiểm tra nhanh bằng cách in ra kiến trúc model
print("\nKiến trúc model:")
print(model)

Đang sử dụng thiết bị: cuda


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([15]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([15, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Model đã được load thành công từ: /content/drive/MyDrive/Colab Notebooks/Final/best_vit_model.pth
Best validation accuracy khi train: 0.8387
Epoch cuối cùng được train: 6

Kiến trúc model:
ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropo

9. BIEU DO TRAIN, LOSS

In [ ]:
import matplotlib.pyplot as plt

# Kiểm tra nếu các biến đã tồn tại trong checkpoint
if 'train_losses' in checkpoint and 'val_losses' in checkpoint and 'train_accs' in checkpoint and 'val_accs' in checkpoint:
    train_losses = checkpoint['train_losses']
    val_losses = checkpoint['val_losses']
    train_accs = checkpoint['train_accs']
    val_accs = checkpoint['val_accs']

    # Vẽ đồ thị
    plt.figure(figsize=(12, 5))

    # Subplot 1: Loss
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Subplot 2: Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='Train Acc')
    plt.plot(val_accs, label='Val Acc')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()
else:
    print("Không tìm thấy training history trong checkpoint. Không thể vẽ đồ thị.")

Không tìm thấy training history trong checkpoint. Không thể vẽ đồ thị.


10. ĐÁNH GIÁ MÔ HÌNH


In [ ]:
test_loss, test_acc, test_preds, test_labels = evaluate(model, test_loader)
print(f'\nTest Accuracy: {test_acc:.4f}')
print('\nClassification Report:')
print(classification_report(test_labels, test_preds, target_names=train_dataset.classes))

11. CONFUSION MATRIX

In [ ]:
cm = confusion_matrix(test_labels, test_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=train_dataset.classes, yticklabels=train_dataset.classes)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
from google.colab import runtime
runtime.save()
print("Đã lưu toàn bộ output vào notebook")
